In [1]:
import sys
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time


In [2]:
year = 2014
# 각 년도 및 기록 웹페이지를 딕셔너리 형태로 저장
# 타격확장 및 투수확장의 경우 규정타석/규정이닝이 아닌 전체타석/전체이닝으로 설정
record_site = {'타격기본':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'타격확장':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=WRCPLUS&o2=WAR_ALL&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수기본':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수확장':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=FIP&o2=WAR&de=0&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=',
'투수구속':f'http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys={year}&ye={year}&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=14&o1=FVval&de=1&o2=WAR&lr=0&tr=&cv=&ml=1&sn=30&si=&cn='
}

In [3]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\105.0.5195.52\chromedriver.exe] found in cache


In [4]:
# 각 페이지를 넘기면서 df를 추출하여 리스트에 추가하기 위해 빈 리스트 생성
driver.get(record_site['타격기본'])
hit_basic_df_list = []
while True:
    page = driver.page_source
    # Statiz 페이지가 두 DataFrame의 리스트 형식으로 되어있어, 두 번째꺼를 추출
    df = pd.read_html(page)[1]
    # 추출한 DataFrame을 hit_basic_df_list에 추가
    hit_basic_df_list.append(df)
    try:
        html = driver.find_element(By.TAG_NAME, 'html')
        next_link = driver.find_element(By.LINK_TEXT, '다음')
        html.send_keys(Keys.END)
        time.sleep(0.5)
        next_link.click()
        time.sleep(0.5)
    except:
        print('수집 종료')
        break

수집 종료


In [5]:
hit_basic_df = pd.concat(hit_basic_df_list)
hit_basic_df

순   이름      팀    정렬    G   타석   타수   득점   안타  2타  ... 희타 희비     비율  \
      순   이름      팀  WAR*    G   타석   타수   득점   안타  2타  ... 희타 희비     타율   
0     1  강정호  14넥SS  8.05  117  501  418  103  149  36  ...  0  2   .356   
1     2  서건창  14넥2B  7.63  128  616  543  135  201  41  ...  4  2   .370   
2     3  테임즈  14N1B  6.32  125  514  443   95  152  30  ...  0  6   .343   
3     4  손아섭  14롯RF  6.26  122  570  483  105  175  25  ...  0  2   .362   
4     5  박병호  14넥1B  6.23  128  571  459  126  139  16  ...  0  4   .303   
..  ...  ...    ...   ...  ...  ...  ...  ...  ...  ..  ... .. ..    ...   
1   302  김민수   14한C -0.69   35   78   74    5   11   1  ...  1  0  0.149   
2   303  고동진  14한RF -0.71   73  213  187   21   45   5  ...  3  3  0.241   
3   304  이태원   14NC -0.71   62   69   66    5    8   2  ...  1  0  0.121   
4   305  이병규  14LDH -0.72   62  207  195   23   49   6  ...  2  0  0.251   
5   306  조정원  14한SS -0.75   20   58   56    2    7   1  ...  1  1  0.125   

                                       WAR*   WPA  
       출루     장타    OPS   wOBA   wRC+  WAR*   WPA  
0    .459   .739  1.198   .500  185.6  8.05  7.25  
1    .438   .547   .985   .437  150.9  7.63  5.80  
2    .422   .688  1.111   .464  168.3  6.32  6.63  
3    .456   .538   .994   .443  154.6  6.26  4.18  
4    .433   .686  1.119   .466  162.9  6.23  4.36  
..    ...    ...    ...    ...    ...   ...   ...  
1   0.182  0.162  0.344  0.165  -29.0 -0.69 -0.78  
2    0.31  0.283  0.593  0.281   45.3 -0.71 -1.49  
3   0.147  0.152  0.299   0.14  -42.9 -0.71 -0.57  
4   0.288  0.313  0.601  0.277   50.3 -0.72 -0.34  
5   0.123  0.179  0.301   0.13  -53.7 -0.75  -0.6  

[346 rows x 31 columns]